# Demand side flexibility


In [31]:
using Pkg
Pkg.add("CSV")
Pkg.add("DataFrames")
Pkg.add("JuMP")
Pkg.add("Gurobi")
#Pkg.add("NLopt")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


In [32]:
using CSV, DataFrames, JuMP, Gurobi

# Read the CSV data

In [33]:
#fixed_cost = Matrix(CSV.read("/Users/adrienguenard/Desktop/MIT/2.AeroAstro/IDS.522/Project/1.WIP/DSF/test_data/fixed_cost.csv",DataFrame))
#variable_cost = Matrix(CSV.read("/Users/adrienguenard/Desktop/MIT/2.AeroAstro/IDS.522/Project/1.WIP/DSF/test_data/variable_cost.csv",DataFrame))
#demand = Matrix(CSV.read("/Users/adrienguenard/Desktop/MIT/2.AeroAstro/IDS.522/Project/1.WIP/DSF/test_data/demand.csv", DataFrame))
#Availability_matrix = Matrix(CSV.read("/Users/adrienguenard/Desktop/MIT/2.AeroAstro/IDS.522/Project/1.WIP/DSF/test_data/availability.csv", DataFrame));

In [34]:
fixed_cost = Matrix(CSV.read("fixed_cost.csv",DataFrame))
variable_cost = Matrix(CSV.read("variable_cost.csv",DataFrame))
demand = Matrix(CSV.read("demand.csv", DataFrame))
max_capacity_matrix = Matrix(CSV.read("max_capacity.csv", DataFrame))
Availability_matrix = Matrix(CSV.read("Availability.csv", DataFrame));

In [35]:
C_FC = fixed_cost[1:4,1]
C_FOM = fixed_cost[1:4,2]
C_VOM = variable_cost[:,1]
E_CO2 = variable_cost[:,2]
X_D = demand[1:1000]
max_capacity = max_capacity_matrix[:]
Availability = Availability_matrix[1:1000,1:2]

1000×2 Matrix{Float64}:
 0.00393802   0.53339
 0.0          0.513005
 0.0          0.518819
 0.0          0.481426
 0.0          0.475112
 0.0          0.469125
 0.0          0.468391
 0.0          0.453982
 0.0          0.411274
 0.0          0.376903
 0.0          0.356235
 0.0          0.375655
 0.0          0.403653
 ⋮            
 0.0          0.431348
 0.0          0.43697
 0.0          0.433476
 0.0          0.420068
 0.0          0.352161
 0.0          0.35399
 0.0          0.370464
 0.0          0.382713
 0.000379575  0.394844
 0.0332202    0.42464
 0.117423     0.459098
 0.229411     0.499756

In [36]:
println(C_FC)

[1.623186399e6, 666821.2034, 1.1105e6, 3.415e6]


# Constant

In [37]:
eta = 0.8# Efficiency of the battery



0.8

# Model Functions

In [38]:
function cost_mix(P, X)
    CO2_price = 1000000 # price of the CO2 per ton
    # The generation for the fossil fuel is for the index 3 and 4 of X
    print(sum((C_FC[i] + C_FOM[i])*P[i] for i in 1:size(P,1)))
    cost = sum((C_FC[i] + C_FOM[i])*P[i] for i in 1:size(P,1)) 
                sum(sum((C_VOM[j] + CO2_price * E_CO2[j])* X[t,j+2] for j in 1:2) for t in 1:size(X,1)) 
    return cost
end

cost_mix (generic function with 1 method)

# Creating the model

List of variable 

In [39]:
Power_generation_ls = 1:4 # Solar, Wind, Battery

fossil_fuel_ls = 1:2 # CCGT, Coal

nb_hours_ls = 1:size(X_D,1)

H = 2

2

In [40]:
model = Model(Gurobi.Optimizer)
println("Declare Model")
set_optimizer_attribute(model, "NonConvex", 2)
#model = Model(NLopt.Optimizer)
#set_optimizer_attribute(model, "algorithm", :LD_MMA)

#The installed power of the mix
@variable(model, P[Power_generation_ls] >= 0)


println("constraint Power")
@constraint(model, Solar_power, P[1] <= 0.5*max_capacity[1])
@constraint(model, Wind_power, P[2] <= 0.5*max_capacity[2])

#Generation of solar, wind, coal, CCGT
@variable(model, X_gen[nb_hours_ls, Power_generation_ls ] >= 0)



println("Objective function")
CO2_price = 100
# The cost of the mix assuming a certain carbon cost
@objective(model, Min, sum((C_FC[i] + C_FOM[i])*P[i] for i in 1:size(P,1)) +
sum(sum((C_VOM[j] + CO2_price * E_CO2[j])* X_gen[t,j+2] for j in 1:2) for t in 1:size(X_gen,1)))


# Cannot produce more than the installed power
println("Installed power constraint")
@constraint(model, Solar_generation[t in nb_hours_ls], X_gen[t,1] <= P[1])
@constraint(model, Wind_generation[t in nb_hours_ls], X_gen[t,2] <= P[2])
@constraint(model, CCGT[t in nb_hours_ls], X_gen[t,3] <= P[3])
@constraint(model, Coal[t in nb_hours_ls], X_gen[t,4] <= P[4])

# Cannot produce more than availability for solar and wind 
println("Availability Constraint")
@constraint(model, Solar_available[t in nb_hours_ls], X_gen[t,1] <= Availability[t,1]*P[1])
@constraint(model, Wind_available[t in nb_hours_ls], X_gen[t,2] <=  Availability[t,2]*P[2])

# Supply meets demand at each hours:
println("Supply meets demand constraint")
@constraint(model, Supply_demand[t in nb_hours_ls], sum(X_gen[t,i] for i in Power_generation_ls) >= X_D[t]);


Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-07
Declare Model


Set parameter NonConvex to value 2
constraint Power
Objective function
Installed power constraint
Availability Constraint
Supply meets demand constraint


In [41]:
optimize!(model);

Set parameter NonConvex to value 2
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 7002 rows, 4004 columns and 15558 nonzeros
Model fingerprint: 0x9333394f
Coefficient statistics:
  Matrix range     [1e-08, 1e+00]
  Objective range  [5e+04, 4e+06]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+05, 5e+07]
Presolve removed 890 rows and 444 columns
Presolve time: 0.00s
Presolved: 6112 rows, 3560 columns, 13780 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   7.087615e+07   0.000000e+00      0s
    1472    2.3324488e+12   0.000000e+00   0.000000e+00      0s

Solved in 1472 iterations and 0.01 seconds (0.02 work units)
Optimal objective  2.332448777e+12

User-callback calls 1516, time in user-callback 0.00 sec


In [42]:
objective_value(model)

2.332448777336431e12

In [43]:
println(value.(P))

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 1:4
And data, a 4-element Vector{Float64}:
      0.0
      2.3680043370160405e6
 317403.2300145676
      0.0


In [44]:
#println(value.(X_gen))
X_gen_matrix = value.(X_gen)




2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, 1:1000
    Dimension 2, 1:4
And data, a 1000×4 Matrix{Float64}:
 0.0       4.57729e5  0.0  0.0
 0.0       4.48949e5  0.0  0.0
 0.0  444216.0        0.0  0.0
 0.0       4.40378e5  0.0  0.0
 0.0       4.42357e5  0.0  0.0
 0.0       4.50506e5  0.0  0.0
 0.0       4.66266e5  0.0  0.0
 0.0       4.69709e5  0.0  0.0
 0.0       4.70139e5  0.0  0.0
 0.0       4.76335e5  0.0  0.0
 0.0       4.8284e5   0.0  0.0
 0.0       4.89425e5  0.0  0.0
 0.0       4.94409e5  0.0  0.0
 ⋮                         
 0.0       5.15446e5  0.0  0.0
 0.0       5.39505e5  0.0  0.0
 0.0       5.68763e5  0.0  0.0
 0.0       5.89897e5  0.0  0.0
 0.0       5.99478e5  0.0  0.0
 0.0       6.01964e5  0.0  0.0
 0.0       6.05141e5  0.0  0.0
 0.0       6.02652e5  0.0  0.0
 0.0       5.96812e5  0.0  0.0
 0.0       5.87828e5  0.0  0.0
 0.0       5.85802e5  0.0  0.0
 0.0       5.91642e5  0.0  0.0

In [45]:
using Plots

@userplot StackedArea

# a simple "recipe" for Plots.jl to get stacked area plots
# usage: stackedarea(xvector, datamatrix, plotsoptions)
@recipe function f(pc::StackedArea)
    x, y = pc.args
    n = length(x)
    y = cumsum(y, dims=2)
    seriestype := :shape

    # create a filled polygon for each item
    for c=1:size(y,2)
        sx = vcat(x, reverse(x))
        sy = vcat(y[:,c], c==1 ? zeros(n) : reverse(y[:,c-1]))
        @series (sx, sy)
    end
end

a = X_gen_matrix[:,1]
b = X_gen_matrix[:,2]
c = X_gen_matrix[:,3]
d = X_gen_matrix[:,4]
sNames = ["Solar";"Wind";"CCGT";"Coal"]
x = nb_hours_ls

plotly()
stackedarea(x, [a b c d], labels=["Solar" "Wind" "CCGT" "Coal"])
#genplot(x, X_D[:])

In [46]:
P_3_matrix = value(P[3])*ones(size(nb_hours_ls))

println(P_3_matrix)
println("X gen")
X_gen_non_battery = sum(X_gen_matrix[:,i] for i =1:2) - X_D[:]
println("X soc")
X_soc_charge = (ones(size(X_soc))- value.(X_soc))* value(P[3]) * H

println("X ch")
X_charge = value.(X_ch)

plot(x, [P_3_matrix X_gen_non_battery X_soc_charge X_charge], label=["P[3]" "X_gen Non batt" "X possible charge" "X_ch"])


[317403.2300145676, 317403.2300145676, 317403.2300145676, 317403.2300145676, 317403.2300145676, 317403.2300145676, 317403.2300145676, 317403.2300145676, 317403.2300145676, 317403.2300145676, 317403.2300145676, 317403.2300145676, 317403.2300145676, 317403.2300145676, 317403.2300145676, 317403.2300145676, 317403.2300145676, 317403.2300145676, 317403.2300145676, 317403.2300145676, 317403.2300145676, 317403.2300145676, 317403.2300145676, 317403.2300145676, 317403.2300145676, 317403.2300145676, 317403.2300145676, 317403.2300145676, 317403.2300145676, 317403.2300145676, 317403.2300145676, 317403.2300145676, 317403.2300145676, 317403.2300145676, 317403.2300145676, 317403.2300145676, 317403.2300145676, 317403.2300145676, 317403.2300145676, 317403.2300145676, 317403.2300145676, 317403.2300145676, 317403.2300145676, 317403.2300145676, 317403.2300145676, 317403.2300145676, 317403.2300145676, 317403.2300145676, 317403.2300145676, 317403.2300145676, 317403.2300145676, 317403.2300145676, 317403.2300

LoadError: UndefVarError: `X_soc` not defined

In [ ]:
println(value.(min_extra_prod_P5_soc))
println(value.(Z_X_gen_lower_X_D))

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 1:1000
And data, a 1000-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.